In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
from DataObjects.article import Article
from DataObjects.legislation import Legislation
from DataObjects.public_consultation import PublicConsultation
from DataObjects.base import Base
from typing import Any
import functools
import requests
import re
from sqlalchemy import create_engine, select, union_all, func, delete, and_
from sqlalchemy.orm import Session
from typing import Callable
from bs4 import BeautifulSoup
import os
from configparser import ConfigParser
from datetime import datetime
config = ConfigParser()
config.read("config.ini")

['config.ini']

In [2]:
engine = create_engine(config.get('DEFAULT', 'db_file'))

In [3]:
def compose(*functions:Callable[...,Any]) -> Callable[...,Any]:
    """ Helper Function to chain the output of a function to another function as Input
    """

    return functools.reduce(lambda f,g: lambda x: g(f(x)), functions )

def extract_text_inside_quoations(text:str) -> str:
    pattern = r'«(.*?)»'
    matches = re.search(pattern=pattern,string=text)
    if matches:
        return matches.group(1).strip()
    else:
        return text.strip()

def extract_text_after_colon(text:str) ->str:
    pattern = ".+:\s?(.+)"
    matches = re.search(pattern=pattern, string=text)
    if matches:
        return matches.group(1).strip()
    else:
        return text.strip()

def remove_part_of_string(text:str, part_of_string:str) -> str:
    pattern = part_of_string+r"\s\S+(.+)"
    matches = re.match(pattern=pattern, string=text)
    if matches:
        return matches.group(1).strip()
    else:
        return text.strip()

def get_first_n_words(text:str, n:int = 4) -> str:
    return " ".join(text.split(" ")[:n])


In [10]:
phrases_to_skip =[
    "ολοκλήρωση της διαδικασίας",
    "ολοκλήρωση της δημόσιας διαβούλευσης επί",
    "δημόσια διαβούλευση για",
    "δημόσια διαβούλευση επί",
    "ρυθμίσεις για",
    "ρυθμισεις για"
    "ενσωμάτωση της οδηγίας",
    "επείγουσες ρυθμίσεις",
    "ολοκλήρωση δημοσίευσης ηλεκτρονικής δημόσιας διαβούλευσης",
    "ολοκλήρωση δημοσίευσης ηλεκτρονικής δημόσιας",
    "ολοκλήρωση διαβούλευσης",
    "δημόσια ηλεκτρονική διαβούλευση",
    "ανοιχτή δημόσια διαβούλευση",
    "κύρωση της σύμβασης",
    "τεχνικές προδιαγραφές για"

] # create a function for string using partial and compose
pre_process_title_fn = str.lower
pre_process_title_fn = compose(pre_process_title_fn,extract_text_inside_quoations)
pre_process_title_fn = compose(pre_process_title_fn,extract_text_inside_quoations)
pre_process_title_fn = compose(pre_process_title_fn,extract_text_after_colon)
for phrase in phrases_to_skip:
    pre_process_title_fn= compose(pre_process_title_fn,functools.partial(remove_part_of_string,part_of_string=phrase))

pre_process_title_fn = compose(pre_process_title_fn,get_first_n_words)

Test Functions

In [11]:
text = "Δημόσια Διαβούλευση για το Σχέδιο Νόμου «Χορήγηση ενισχύσεων σε ιδιωτικές επενδύσεις για την προώθηση της οικονομικής και κοινωνικής ανάπτυξης και την περιφερειακή σύγκλιση και άλλες διατάξεις»"
text = extract_text_inside_quoations(text) 
print(text)
text = get_first_n_words(text)
print(text)


text = "‘Δημόσια Διαβούλευση για τις δράσεις «Digi-content» και «Digi-retail»"
text = extract_text_after_colon(text) 
print(text)
text = remove_public_consultation_string(text)
print(text)
text = get_first_n_words(text)
print(text)

Χορήγηση ενισχύσεων σε ιδιωτικές επενδύσεις για την προώθηση της οικονομικής και κοινωνικής ανάπτυξης και την περιφερειακή σύγκλιση και άλλες διατάξεις
Χορήγηση ενισχύσεων σε ιδιωτικές
‘Δημόσια Διαβούλευση για τις δράσεις «Digi-content» και «Digi-retail»
‘Δημόσια Διαβούλευση για τις δράσεις «Digi-content» και «Digi-retail»
‘Δημόσια Διαβούλευση για τις


In [12]:
# pre_process_title_fn = compose(str.lower,
#                                extract_text_inside_quoations,
#                                extract_text_after_colon,
#                                remove_public_consultation_string,
#                                remove_public_consultation_completed_string,
#                                remove_regularization_for_string,
#                                remove_incorporation_of_european_guidance_string,
#                                remove_urgent_regularization_for_string,
#                                get_first_n_words)

text = "Δημόσια Διαβούλευση για το Σχέδιο Νόμου «Χορήγηση ενισχύσεων σε ιδιωτικές επενδύσεις για την προώθηση της οικονομικής και κοινωνικής ανάπτυξης και την περιφερειακή σύγκλιση και άλλες διατάξεις»"
text = pre_process_title_fn(text)
print(text)
text2 = "Σχέδιο Νόμου: Ενίσχυση Κοινωνικής Αλληλεγγύης και  Εισφορά Κοινωνικής Ευθύνης"
text2 = pre_process_title_fn(text2)
print(text2)
txt3 = "Ολοκλήρωση της δημόσιας διαβούλευσης επί του Εθνικού Σχεδίου Δράσης για την Ψυχική Υγεία 2021-2030"
txt3 = pre_process_title_fn(txt3)
print(txt3)

txt4 = "Διαβούλευση επί του σχεδίου νόμου «Οργανισμοί Διαχείρισης και Προώθησης Προορισμού, Ιαματικές Πηγές Ελλάδας, Ναυάγιο Ζακύνθου, διατάξεις για τα τουριστικά γραφεία, τις τουριστικές επιχειρήσεις και τα τουριστικά καταλύματα και άλλες ρυθμίσεις για την τουριστική ανάπτυξη»"
txt4 = pre_process_title_fn(txt4)
print(txt4)

χορήγηση ενισχύσεων σε ιδιωτικές
ενίσχυση κοινωνικής αλληλεγγύης και
εθνικού σχεδίου δράσης για
οργανισμοί διαχείρισης και προώθησης


In [14]:
with Session(engine) as sess:
    stmt = select(Legislation.title)
    titles = sess.execute(stmt).scalars()
    for title in titles:
        print(f"Title:{pre_process_title_fn(title)} <- {(title)}")

Title:αρση περιορισμων για την <- Ολοκλήρωση της δημόσιας ηλεκτρονικής διαβούλευσης για το σχέδιο νόμου του Υπουργείου Εσωτερικών με τίτλο «ΑΡΣΗ ΠΕΡΙΟΡΙΣΜΩΝ ΓΙΑ ΤΗΝ ΕΓΓΡΑΦΗ ΣΤΟΥΣ ΕΙΔΙΚΟΥΣ ΕΚΛΟΓΙΚΟΥΣ ΚΑΤΑΛΟΓΟΥΣ ΕΚΛΟΓΕΩΝ ΕΞΩΤΕΡΙΚΟΥ».
Title:οδηγού προσβασιμότητας για ιστοτόπους <- Ανοιχτή δημόσια διαβούλευση του Οδηγού Προσβασιμότητας για ιστοτόπους και εφαρμογές για φορητές συσκευές των οργανισμών της Ελληνικής Δημόσιας Διοίκησης
Title:ρυθμίσεις σχετικά με τους <- Ολοκλήρωση διαβούλευσης για το σχέδιο νόμου με τίτλο «Ρυθμίσεις σχετικά με τους Οργανισμούς Τοπικής Αυτοδιοίκησης α’ και β’ βαθμού – Ρυθμίσεις σχετικά με τον ειδικό εκλογικό χώρο για ΑμεΑ και το δικαίωμα του εκλέγεσθαι – Διατάξεις για την ευζωία των ζώων συντροφιάς – Διατάξεις για το ανθρώπινο δυναμικό του δημοσίου τομέα – Λοιπές ρυθμίσεις του Υπουργείου Εσωτερικών»
Title:φαρμακευτικοι συλλογοι – πανελληνιος <- Δημόσια ηλεκτρονική διαβούλευση για το σχέδιο νόμου του Υπουργείου Υγείας με τον τίτλο «ΦΑΡΜΑΚΕΥΤΙΚΟΙ ΣΥΛΛΟΓΟΙ – ΠΑΝΕΛ

Do actual Look up of Final Legislations

In [15]:
import logging



logging.basicConfig(
    filename="final_legislations.log",
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


SEARCH_URL = "https://www.hellenicparliament.gr/Nomothetiko-Ergo/Anazitisi-Nomothetikou-Ergou"
DOMAIN = "https://www.hellenicparliament.gr"
PDF_PATH ="pdf_files"


with Session(engine) as sess:
    legObjs= sess.execute(
        select(Legislation).where(and_(
            Legislation.id<=880, Legislation.final_legislation_id.is_(None)
        ))
    ).scalars()
    for legObj in legObjs:
        if legObj.final_legislation_id is not None:
            logging.info(f"Legislation Object {LegObj} was already processed with related Legislation id {legObj.final_legislation_id}")
            continue

        title = legObj.title
        title_to_search = pre_process_title_fn(title)
        
        payload = {'criteria':title_to_search}
        r = requests.get(SEARCH_URL,params=payload)

        logging.info(f"Trying url {r.url} for object {legObj.id}, {legObj.title}")

        if r.status_code!=200:
            logging.info(f"Url {r.url} did not respond with 200. Legislation id=={legObj.id}, {legObj.title}")
            continue

        in_search_soup = BeautifulSoup(r.content, features="html.parser")
        if len(in_search_soup.select('tr td a'))>1:
            logging.info(f"Multiple results returned for {legObj.id} with count={len(in_search_soup.select('tr td a'))}, url{r.url}")
        
        if len(in_search_soup.select('tr td a'))==0:
            logging.info(f"Did not find legislation for legObj with id={legObj.id}")
            continue

        legislation_link = in_search_soup.select_one('tr td a').get('href')
        r2 = requests.get(DOMAIN+legislation_link)

        legislation_soup = BeautifulSoup(r2.content, features="html.parser")

        if legislation_soup.find(string="Αριθμός Φεκ") is None: # not a Legislation
            continue

        data={}

        title = legislation_soup.find(string='Τίτλος').next_element.find('dd').string
        data["title"] = title

        legislation_type = legislation_soup.find(string='Τύπος').next_element.find('dd').string if legislation_soup.find(string='Τύπος') is not None else None
        data["legislation_type"] = legislation_type

        ministry = legislation_soup.find(string="Υπουργείο").next_element.find('dd').string if legislation_soup.find(string='Υπουργείο') is not None else None
        data["ministry"] = ministry

        fek_number = legislation_soup.find(string="Αριθμός Φεκ").next_element.find('dd').string if legislation_soup.find(string='Αριθμός Φεκ') is not None else None
        data["fek_number"] = fek_number

        date_posted = legislation_soup.find(string="Ημ. Ψήφισης").next_element.find('dd').string if legislation_soup.find(string='Ημ. Ψήφισης') is not None else None
        date_posted = datetime.strptime(date_posted,"%d/%m/%Y") if date_posted is not None else None
        data["date_posted"] = date_posted

        legislation_url = legislation_soup.find(string="Ψηφισθέν Νομοσχέδιο").next_element.find('a').get('href')
        data["legislation_pdf_url"] = legislation_url

        data["scrap_url"] = r2.url

        newLegObj = Legislation(**data)
        sess.add(newLegObj)
        sess.commit()  # Get Primary Key
        print(newLegObj)
        legObj.final_legislation_id = newLegObj.id
        sess.commit()

        with open(os.path.join(PDF_PATH,f"{str(newLegObj.id)}.pdf"),'wb') as pdffile:
            response = requests.get(legislation_url) 
            pdffile.write(response.content)

  
    
logging.shutdown()

Legislation(id=1178, title=Διατάξεις για τη θωράκιση του θεσμικού πλαισίου του αθλητισμού και τον εξορθολογισμό της αθλητικής νομοθεσίας, ministry=Πολιτισμού και Αθλητισμού, date_posted=2023-02-21)
Legislation(id=1179, title=Κανόνες Τεκμηρίωσης Ενδοομιλικών Συναλλαγών, Κανόνες Υποκεφαλαιοδότησης Επιχειρήσεων, Διαδικασία Ταχείας Αδειοδότησης και άλλες διατάξεις.
ΕΙΣΗΓΗΤΕΣ: Π. Μπετζάλη, Κ. Σπηλιόπουλος, ministry=Οικονομίας και Οικονομικών, date_posted=2009-07-14)
Legislation(id=1180, title=Σύσταση νομικού προσώπου ιδιωτικού δικαίου με την επωνυμία «Ογκολογικό Κέντρο Παίδων ''Μαριάννα Β. 
Βαρδινογιάννη - ΕΛΠΙΔΑ''», εκσυγχρονισμός του δικαίου για τη δωρεά και μεταμόσχευση οργάνων, ρυθμίσεις για 
την αντιμετώπιση της πανδημίας του κορωνοϊού COVID-19 και την προστασία της δημοσίας υγείας και άλλες 
επείγουσες ρυθμίσεις, ministry=Υγείας, date_posted=2023-03-15)
Legislation(id=1181, title=Διαχείριση, έλεγχος  και  εφαρμογή αναπτυξιακών παρεμβάσεων για την προγραμματική περίοδο 2007-2013. 
ΕΙΣΗ

after scraping, compare titles with final and initial legislation
find legislationId with mutiple counts

Logging

In [41]:
import logging

logging.basicConfig(
    filename="app.log",
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logging.debug("This is a debug message")
logging.info("This is an info message")
logging.warning("This is a warning message")
logging.error("This is an error message")
logging.critical("This is a critical message")

logging.shutdown()